In [ ]:
import pandas as pd
import pickle
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

florida = pickle.load(open("../data/florida.pkl", "rb"))
alberta = pickle.load(open("../data/alberta.pkl", "rb"))

frames = [florida, alberta]

data = pd.concat(frames)
print(data)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Microcystin (ug/L)']

florida_x=florida['DATETIME']
florida_y=florida['Microcystin (ug/L)']

layout = go.Layout(
    title="Microcystin vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Microcystin (ug/L)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Total Nitrogen (ug/L)']

florida_x=florida['DATETIME']
florida_y=florida['Total Nitrogen (ug/L)']

layout = go.Layout(
    title="Total Nitrogen vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Total Nitrogen (ug/L)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Total Phosphorus (ug/L)']

florida_x=florida['DATETIME']
florida_y=florida['Total Phosphorus (ug/L)']

layout = go.Layout(
    title="Total Phosphorus vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Total Phosphorus (ug/L)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Secchi Depth (m)']

florida_x=florida['DATETIME']
florida_y=florida['Secchi Depth (m)']

layout = go.Layout(
    title="Secchi Depth vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Secchi Depth (m)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Total Chlorophyll (ug/L)']

florida_x=florida['DATETIME']
florida_y=florida['Total Chlorophyll (ug/L)']

layout = go.Layout(
    title="Total Chlorophyll vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Total Chlorophyll (ug/L)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)

In [ ]:
alberta_x=alberta['DATETIME']
alberta_y=alberta['Temperature (degrees celsius)']

florida_x=florida['DATETIME']
florida_y=florida['Temperature (degrees celsius)']

layout = go.Layout(
    title="Temperature vs Date", 
    xaxis={'title':'Date'}, 
    yaxis={'title':'Temperature (degrees celsius)'}
)

a = go.Scatter(
    x=alberta_x,
    y=alberta_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Alberta'
)

f = go.Scatter(
    x=florida_x,
    y=florida_y,
    mode='markers',
    marker={
   'opacity': 0.6,
    },
    name='Florida'
)

fig = {
    'data': [f, a],
    'layout': layout
}

iplot(fig)